In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dataset
import json
import pickle
import os

In [6]:
!ls /ivi/ilps/personal/avardas/_data/raw/yahoo

binarized_purged.npz  set2.test.txt   set2.valid.txt  train.txt
binarized_purged.pkl  set2.train.txt  test.txt	      valid.txt


In [7]:
!ls /ivi/ilps/personal/avardas/_data/raw/mslr

binarized_purged.npz  binarized_purged.pkl  test.txt  train.txt  valid.txt


In [ ]:
!ls /ivi/ilps/personal/avardas/_data/raw/istella

In [11]:
def make_small(dataset, tr_new_size, va_new_size):
    dataset['trdlr'] = dataset['trdlr'][:tr_new_size]
    dataset['trfm'] = dataset['trfm'][:dataset['trdlr'][-1], :]
    dataset['trlv'] = dataset['trlv'][:dataset['trdlr'][-1]]

    dataset['vadlr'] = dataset['vadlr'][:va_new_size]
    dataset['vafm'] = dataset['vafm'][:dataset['vadlr'][-1], :]
    dataset['valv'] = dataset['valv'][:dataset['vadlr'][-1]]

def save_pkl(dataset, pkl_path, tr_new_size=-1, va_new_size=-1):
    ds = {}
    ds['trfm'] = dataset.train.feature_matrix
    ds['tefm'] = dataset.test.feature_matrix
    ds['vafm'] = dataset.valid.feature_matrix
    ds['trlv'] = dataset.train.label_vector
    ds['telv'] = dataset.test.label_vector
    ds['valv'] = dataset.valid.label_vector
    ds['trdlr'] = dataset.train.doclist_ranges
    ds['tedlr'] = dataset.test.doclist_ranges
    ds['vadlr'] = dataset.valid.doclist_ranges
    if tr_new_size > 0:
        make_small(ds, tr_new_size, va_new_size)
    with open(pkl_path, 'wb') as f:
        pickle.dump(ds, f)

In [ ]:
with open('datasets_info.json', 'r') as f:
    ds_info = json.load(f)
    
    
for name in ['mslr', 'yahoo']:
    data = dataset.get_dataset_from_json_info(
                    name,
                    'datasets_info.json',
                    feature_normalization=False
                  ).get_data_folds()[0]
    data.read_data()
    print(name, data.train.doclist_ranges.shape, data.valid.doclist_ranges.shape)
    save_pkl(data, os.path.join(ds_info[name]['fold_paths'][0], 'binarized_purged_small.pkl'), 1000, 500)

In [4]:
with open('../ltr/datasets_info.json', 'r') as f:
        datasets_info = json.load(f)

    
def read_datatset(datasets_info, name):
    with open(datasets_info[name]['path'], 'rb') as f:
        dataset = pickle.load(f)
    with open(datasets_info[name]['path'].replace('.pkl', datasets_info[name]['group_suffix']), 'rb') as f:
        groups = pickle.load(f)
    return type('ltr', (object, ), dataset), groups

dataset, all_groups = read_datatset(datasets_info, 'yahoo')

In [6]:
dataset.telv[:100]

array([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 2, 2, 0, 1, 2, 1, 2, 1, 2, 0, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 3, 1, 2, 0, 3, 1, 2, 1, 2, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 3, 0, 0,
       2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])